### Expected Goals including player positions
Step-by-step walk thru of making an expected goals model leveraging additional information concerning opposition player location. This tutorial follows similar design choices as Javier Fernandez’s expected goals model in "A framework for the fine-grained evaluation of the instantaneous expected value of soccer possessions.

This effort will train a shallow neural network with following features:

- ball location (x)
- binary variable signifying if ball was closer to the goal than the opponent’s goalkeeper
- angle between the ball and the goal
- distance between the ball and the goal
- distance between the ball and the goalkeeper in y-axis
- distance between the ball and the goalkeeper
- number of opponent players inside the triangle formed between the ball location and opponent’s goal posts
- number of opponent players less than 3 meters away from the ball location
- binary variable signifying if shot was a header
- expected goals based on distance to goal and angle between the ball and the goal

In [1]:
#importing necessary libraries
from mplsoccer import Sbopen
import pandas as pd
import numpy as np
import warnings
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import os
import random as rn
import tensorflow as tf
#warnings not visible on the course webpage
pd.options.mode.chained_assignment = None
warnings.filterwarnings('ignore')

2023-12-10 20:59:02.924658: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
#setting random seeds so that the results are reproducible on the webpage

os.environ['PYTHONHASHSEED'] = '0'
os.environ['CUDA_VISIBLE_DEVICES'] = ''
np.random.seed(1)
rn.seed(1)
tf.random.set_seed(1)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

### Opening data
This task will use Statsbomb Indian Super League 2021/2022 data since it is the only dataset openly available that **contains both event and tracking data** for the entire season. Open each game and store data for the entire season in dataframes shot_df and track_df. Then change yards to meters. In the end, filter open play shots and remove shots without the goalkeeper tracked.

In [3]:
parser = Sbopen()
#get list of games during Indian Super League season
df_match = parser.match(competition_id=1238, season_id=108)
matches = df_match.match_id.unique()
matches

array([3827767, 3827335, 3827336, 3827338, 3827337, 3817856, 3817891,
       3817870, 3817899, 3817866, 3817889, 3817887, 3817872, 3817900,
       3817855, 3817863, 3817877, 3817901, 3817880, 3817902, 3817862,
       3817873, 3817869, 3817881, 3817895, 3817884, 3817896, 3817852,
       3817883, 3817886, 3817864, 3817882, 3817894, 3817871, 3817861,
       3817888, 3817875, 3817868, 3817867, 3813313, 3817890, 3813302,
       3817857, 3817898, 3817893, 3817854, 3817892, 3817859, 3817858,
       3817897, 3817885, 3817878, 3817876, 3813305, 3813266, 3813271,
       3813311, 3813303, 3813304, 3813306, 3813283, 3813275, 3813278,
       3813295, 3813296, 3813282, 3813269, 3817879, 3817874, 3817865,
       3817860, 3817853, 3817851, 3817850, 3817849, 3817848, 3813318,
       3813287, 3813270, 3813307, 3813279, 3813281, 3813274, 3813285,
       3813317, 3813316, 3813315, 3813314, 3813312, 3813310, 3813309,
       3813308, 3813301, 3813300, 3813299, 3813298, 3813297, 3813294,
       3813293, 3813

In [4]:
shot_df = pd.DataFrame()
track_df = pd.DataFrame()
#store data in one dataframe
for match in matches:
    shots = (parser.event(match)[0] # open events
             .query("type_name == 'Shot'") # query shots
             .assign(x = lambda df: df.x.apply(lambda cell: cell*105/120), # assign column updates
                     y = lambda df: df.y.apply(lambda cell: cell*68/80))
             )
    
    df_track = (parser.event(match)[2] # open 360 data
                .assign(x = lambda df: df.x.apply(lambda cell: cell*105/120), # assign column updates
                        y = lambda df: df.y.apply(lambda cell: cell*68/80)))
    
    #append event and trackings to a dataframe
    shot_df = pd.concat([shot_df, shots], ignore_index = True)
    track_df = pd.concat([track_df, df_track], ignore_index = True)
# reset indicies
shot_df.reset_index(drop=True, inplace=True)
track_df.reset_index(drop=True, inplace=True)

# filter out non open-play shots
shot_df = shot_df.query('sub_type_name == "Open Play"')

#filter out shots where goalkeeper was not tracked
gks_tracked = track_df.query("teammate == False and position_name == 'Goalkeeper'")['id'].unique()
    # gks_tracked = track_df.loc[track_df["teammate"] == False].loc[track_df["position_name"] == "Goalkeeper"]['id'].unique()
shot_df = shot_df.loc[shot_df["id"].isin(gks_tracked)]

In [8]:
shot_df.sample(3)

,id,index,period,timestamp,minute,second,possession,duration,match_id,type_id,...,shot_deflected,shot_open_goal,ball_recovery_offensive,pass_miscommunication,player_off_permanent,dribble_no_touch,foul_committed_penalty,foul_won_penalty,shot_follows_dribble,shot_redirect
2160,9bbb8185-7062-43a0-8799-3152959cf926,896,1,00:22:10,22,10,54,1.158561,3813317,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1397,9f21f417-17ba-45c9-b7de-4be95d449909,3120,2,00:52:55,97,55,197,1.183715,3817876,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1723,7834afb5-e505-4028-9307-0778f636bb22,2383,2,00:27:05,72,5,154,0.243436,3813269,16,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
track_df.sample(3)

,teammate,match_id,id,x,y,player_id,player_name,position_id,position_name,event_freeze_id
17854,False,3817892,0821a975-2be4-4d8d-97ee-942d4bf2206e,87.7625,23.715,24860,Héctor Rodas Ramírez,3,Right Center Back,8
5948,False,3817863,4763d971-6861-4e3d-a862-ae6953ebb7e5,95.2000,23.800,124903,Akash Mishra,6,Left Back,7
40295,False,3813268,058bf268-4f99-405d-85a9-a5a5eba780cf,87.0625,43.435,124756,Subhasish Bose,8,Left Wing Back,3


In [10]:
shot_df.to_csv('shot_df_xGpos.csv')
track_df.to_csv('track_df_xGpos.csv')

### Feature engineering
This section creates features to improve the dataset. They will be stored in a model_vars dataframe. It is suggested to read the code comments to understand this part of tutorial better.  The details are embedded within the code.